# Corona Project


In [1]:
import numpy as np 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import json
import plotly.express as px

In [2]:
url = "https://raw.githubusercontent.com/oczerwia/corona_map/main/landkreise_simplify20.geojson"
NRW_map =  requests.get(url)
NRW_map = NRW_map.text
NRW_data = json.loads(NRW_map)
geo_df = gpd.read_file(url)
geo_df.crs = {'init': 'epsg:4326'}

In [3]:
RKI_url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=BL%20%3D%20%27NORDRHEIN-WESTFALEN%27&outFields=OBJECTID,ADE,GF,BSG,RS,AGS,SDV_RS,GEN,BEZ,IBZ,BEM,NBD,SN_L,SN_R,SN_K,SN_V1,SN_V2,SN_G,FK_S3,NUTS,RS_0,AGS_0,WSK,EWZ,KFL,DEBKG_ID,death_rate,cases,deaths,cases_per_100k,cases_per_population,BL,BL_ID,county,last_update,cases7_per_100k,recovered,EWZ_BL,cases7_bl_per_100k&outSR=4326&f=json"
RKI = requests.get(RKI_url)
RKI = RKI.text
RKI_data = json.loads(RKI)

In [4]:
RKI_dataframe = pd.DataFrame(columns = ["Name", "cases","cases_per_100k","deaths"])
RKI_dataframe["Name"] = [name["attributes"]["GEN"] for name in RKI_data["features"]]
RKI_dataframe["cases"] = [name["attributes"]["cases"] for name in RKI_data["features"]]
RKI_dataframe["cases_per_100k"] = [name["attributes"]["cases7_per_100k"] for name in RKI_data["features"]]
RKI_dataframe["deaths"] = [name["attributes"]["deaths"] for name in RKI_data["features"]]

In [5]:
population_df = pd.DataFrame(columns = ["Name", "population"])
population_df["Name"] = [cityname["properties"]["GEN"] for cityname in NRW_data["features"]]
population_df["population"] = [population["properties"]["destatis"]["population"] for population in NRW_data["features"]]

In [6]:
full_stats = RKI_dataframe.merge(population_df, how= "inner",on ="Name")
full_stats.head()

,Name,cases,cases_per_100k,deaths,population
0,Düsseldorf,9062,160.964306,89,619294
1,Duisburg,9353,268.505633,133,498590
2,Essen,7020,146.544032,76,583109
3,Krefeld,3146,159.178953,36,227020
4,Mönchengladbach,3161,190.013561,65,261454


In [7]:
geo_df = geo_df.rename(columns={'geometry': 'geometry','GEN':'Name'}).set_geometry('geometry')
geo_df.drop(columns = ["ADE", "GF", "BSG", "RS", "AGS", "SDV_RS","BEZ", "IBZ", "BEM", "SN_V2", "SN_G", "FK_S3", "NUTS",
                       "RS_0", "AGS_0", "WSK", "DEBKG_ID","NBD", "SN_L", "SN_R", "SN_K", "SN_V1", "destatis"], inplace = True)

In [8]:
full_stats = geo_df.merge(full_stats, how = "inner", on = "Name")

In [9]:
full_stats.head()

,Name,geometry,cases,cases_per_100k,deaths,population
0,Düsseldorf,"POLYGON ((6.75294 51.34988, 6.75306 51.34945, ...",9062,160.964306,89,619294
1,Duisburg,"POLYGON ((6.70300 51.55403, 6.70729 51.55077, ...",9353,268.505633,133,498590
2,Essen,"POLYGON ((7.01895 51.52054, 7.02071 51.51806, ...",7020,146.544032,76,583109
3,Krefeld,"POLYGON ((6.54789 51.40013, 6.54496 51.39660, ...",3146,159.178953,36,227020
4,Mönchengladbach,"POLYGON ((6.47750 51.23659, 6.47813 51.23580, ...",3161,190.013561,65,261454
